In [1]:
import os 
import sys
import pandas as pd
import numpy as np
from multiprocessing import cpu_count

# Data preparation

In [2]:
files = ["/home/aazia/Desktop/projects/IMU_LSTM/data/data/1/annotations.txt","/home/aazia/Desktop/projects/IMU_LSTM/data/data/2/annotations.txt","/home/aazia/Desktop/projects/IMU_LSTM/data/data/3/annotations.txt","/home/aazia/Desktop/projects/IMU_LSTM/data/data/4/annotations.txt","/home/aazia/Desktop/projects/IMU_LSTM/data/data/5/annotations.txt"]
big_Data = []
c=1
for file in files:
    _col = []
    df = pd.read_csv(file, sep="\s", header=None, names=['start', 'end', 'value'])
    for x in range(len(df["start"])):
        start = df["start"][x]
        end = df["end"][x]
        for y in range(start,end):
            _col.append([y ,df["value"][x]])
    df3 = pd.DataFrame(_col, columns=["index", "value"])
    df3.to_csv("big"+str(c))
    c=c+1
    big_Data.append(_col)
    

/home/arsalan/main-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
base_path = "/home/aazia/Desktop/projects/IMU_LSTM/data/data"
dir = os.listdir(base_path)
file_list = []
for each in dir:
    file_list.append(os.path.abspath(str((os.path.join(base_path,each)))))
file_list.sort()

In [4]:
data = pd.DataFrame()
c=1
for each in file_list:
    file = os.listdir(each)
    file.sort()
    for f in file:
        if f.endswith("csv"):
            r = os.path.join(each,f)
            df = pd.read_csv(r, sep=",")
            df.reset_index(inplace=True)
            df.to_csv("df"+str(f)+str(c))
            data = data.append(df)
    c=c+1
            
            

In [5]:
data.head()

,index,time,ax,ay,az,gx,gy,gz
0,0,1595877718,0.2382,-0.9150,0.1859,-0.3049,-0.4268,-0.3659
1,1,1595877718,0.2427,-0.9233,0.1884,-0.2439,-0.7317,-0.3049
2,2,1595877718,0.2432,-0.9219,0.1906,-0.1220,-0.9146,0.0000
3,3,1595877718,0.2446,-0.9222,0.1888,-0.1829,-0.7317,0.1220
4,4,1595877718,0.2470,-0.9247,0.1887,-0.2439,-0.6707,0.1829


In [6]:
main_df = pd.DataFrame()
for each in range(1,6):
   
    df1 = pd.read_csv("big"+str(each), index_col=0)
    for x in range(0,3):
        df2 = pd.read_csv("dfimu_"+str(x)+".csv"+str(each), index_col=0)
        frame = df2.merge(df1, on='index',how='left')
        main_df = main_df.append(frame)
       

In [7]:
main_df = main_df.dropna()
main_df.reset_index(inplace=True)
main_df = main_df.drop(columns=["time","index", "level_0"], axis=1)
main_df.head()

,ax,ay,az,gx,gy,gz,value
0,0.7902,0.4138,0.6031,15.1220,1.0976,-13.5366,get_a_b
1,0.7663,0.4313,0.5903,15.7927,-1.4024,-13.7805,get_a_b
2,0.7838,0.4391,0.5827,16.6463,-7.1341,-13.8415,get_a_b
3,0.8204,0.4349,0.5819,16.9512,-11.7683,-13.4146,get_a_b
4,0.8328,0.4305,0.5737,16.8293,-12.7439,-13.1098,get_a_b


In [8]:
main_df.head()

,ax,ay,az,gx,gy,gz,value
0,0.7902,0.4138,0.6031,15.1220,1.0976,-13.5366,get_a_b
1,0.7663,0.4313,0.5903,15.7927,-1.4024,-13.7805,get_a_b
2,0.7838,0.4391,0.5827,16.6463,-7.1341,-13.8415,get_a_b
3,0.8204,0.4349,0.5819,16.9512,-11.7683,-13.4146,get_a_b
4,0.8328,0.4305,0.5737,16.8293,-12.7439,-13.1098,get_a_b


In [9]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115770 entries, 0 to 115769
Data columns (total 7 columns):
ax       115770 non-null float64
ay       115770 non-null float64
az       115770 non-null float64
gx       115770 non-null float64
gy       115770 non-null float64
gz       115770 non-null float64
value    115770 non-null object
dtypes: float64(6), object(1)
memory usage: 6.2+ MB


# LSTM MODEL

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler

In [11]:
def create_datasets(X, y, test_size=0.2, time_dim_first=False):
    enc = LabelEncoder()
    y_enc = enc.fit_transform(y)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y_enc, test_size=0.1)
    X_train, X_valid = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train.values, X_valid.values)]
    y_train, y_valid = [torch.tensor(arr, dtype=torch.long) for arr in (y_train, y_valid)]
    train_ds = TensorDataset(X_train, y_train)
    valid_ds = TensorDataset(X_valid, y_valid)
    return train_ds, valid_ds, enc

In [12]:
def create_loaders(train_ds, valid_ds, bs=512, jobs=0):
    train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=jobs)
    valid_dl = DataLoader(valid_ds, bs, shuffle=False, num_workers=jobs)
    return train_dl, valid_dl
def accuracy(output, target):
    return (output.argmax(dim=1) == target).float().mean().item()

In [13]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        x = x.view(-1,1,6)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t for t in (h0, c0)]

In [14]:
y = main_df["value"]
feature = main_df.drop("value", axis=1)

print('Preparing datasets')
trn_ds, val_ds, enc = create_datasets(feature, y)

Preparing datasets


In [15]:
y.value_counts()

screw             48896
get_a_b           16140
get_tools         14688
hold_for_robot    13089
attach_a_b         9285
insert_screws      7286
attach_a_b_c       6386
Name: value, dtype: int64

In [16]:
bs = 128
print(f'Creating data loaders with batch size: {bs}')
trn_dl, val_dl = create_loaders(trn_ds, val_ds, bs, jobs=cpu_count())

Creating data loaders with batch size: 128


In [17]:
input_dim = 6   
hidden_dim = 256
layer_dim = 3
output_dim = 7
seq_dim = 128

lr = 0.005
n_epochs = 200
iterations_per_epoch = len(trn_dl)
best_acc = 0
patience, trials = 100, 0

model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1): # loop over the dataset multiple times
    running_loss = 0.0
    for i,  (x_batch, y_batch)  in enumerate(trn_dl, 0):
        model.train()
        opt.zero_grad()
        out = model(x_batch)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    model.eval()
    correct, total = 0, 0
    for x_val, y_val in val_dl:
        x_val, y_val = [t for t in (x_val, y_val)]
        out = model(x_val)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()
    
    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch 1 best model saved with accuracy: 52.54%
Epoch 2 best model saved with accuracy: 57.96%
Epoch 3 best model saved with accuracy: 61.10%
Epoch 4 best model saved with accuracy: 61.53%
Epoch:   5. Loss: 0.2982. Acc.: 63.96%
Epoch 5 best model saved with accuracy: 63.96%
Epoch 8 best model saved with accuracy: 67.16%
Epoch 9 best model saved with accuracy: 67.70%
Epoch:  10. Loss: 0.7420. Acc.: 66.90%
Epoch 14 best model saved with accuracy: 69.34%
Epoch:  15. Loss: 0.0136. Acc.: 70.45%
Epoch 15 best model saved with accuracy: 70.45%
Epoch 17 best model saved with accuracy: 71.73%
Epoch:  20. Loss: 0.1278. Acc.: 72.48%
Epoch 20 best model saved with accuracy: 72.48%
Epoch:  25. Loss: 1.1107. Acc.: 72.68%
Epoch 25 best model saved with accuracy: 72.68%
Epoch 27 best model saved with accuracy: 73.27%
Epoch:  30. Loss: 0.2732. Acc.: 72.85%
Epoch 32 best model saved with accuracy: 73.95%
Epoch:  35. Loss: 0.0200. Acc.: 73.86%
Epoch 37 best model saved with accuracy: 